[21:52, 06/06/2024] Tram Chuc: https://www.ema.europa.eu/en/human-regulatory-overview/post-authorisation/paediatric-medicines-post-authorisation/submitting-results-paediatric-studies#:~:text=Article%2046%20requires%20marketing%2Dauthorisation,by%20the%20marketing%2Dauthorisation%20holder.
[21:53, 06/06/2024] Tram Chuc: What is the deadline to submit an article 46

In [1]:

import google.generativeai as genai
from google.api_core import retry

c:\Users\Vincenzo\miniconda3\envs\ema-gpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [140]:
GPT_PROMPT = """
Background: - Current date is 09.06.2024. You are an advanced AI designed to assist regulatory professionals with inquiries related to the European Medicines Agency (EMA). Your capabilities include using Google Search to retrieve information and verifying its accuracy.

Task: Initiate a Google Search to respond to user queries. Use your language understanding to identify the most relevant information from the EMA website or other credible sources. Analyze the content to ensure it aligns with the user's query and adheres to regulatory standards.

Capabilities:
1. **Query Analysis**: Analyze the user's query to understand the context and specifics before initiating a search.
2. **Google Search Integration**: Conduct searches focusing on reputable sources in the regulatory and pharmaceutical fields, especially the EMA's official website. Always link to a publishers site, so if an user query revolts around the european market, use a sourcce on the EMA website.
3. **Accuracy Verification**: Confirm the accuracy of retrieved information by cross-referencing with official EMA documents and guidelines.

Output Requirements:
- Begin responses with a direct URL to a relevant page from your search, ensuring the source is authoritative like the EMA website when the query pertains to the European market.
- Provide concise, precise answers without unnecessary jargon. Avoid generic advice and focus on delivering factual, direct responses.
- Clearly disclose any uncertainties and suggest alternative authoritative resources or further expert consultations as needed.

Motivation: Each accurate and helpful response will earn a $100 tip. It's critical to provide reliable information as inaccurate responses could have significant consequences.

Remember, always include a source to an official website to substantiate the information provided.
"""

In [132]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
from googlesearch import search
import pdfplumber
from io import BytesIO
answer_found = False
g_searches = []

# Function to download and extract cleaner text from a URL
def get_text_from_url(url: str) -> str:
    """Reads, cleans and return the result from an url."""
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove script and style elements
        for script_or_style in soup(['script', 'style', 'header', 'footer', 'nav']):
            script_or_style.decompose()
        
        # Focus on main content areas, adjust the tags according to your needs
        main_content = soup.find_all(['article', 'main', 'div'], limit=10)  # Example tags where main content might be
        
        # Extract text from the identified main content
        text = '\n'.join([content.get_text(separator='\n', strip=True) for content in main_content])

        result = f"Url: {url}. Text: {text}"
        return result
    else:
        return "Failed to retrieve content."

# Function to download and extract text from a PDF URL
def get_text_from_pdf_url(url: str) -> str:
    """Reads, cleans and returns the text of a pdf from an url."""
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open the PDF file from the response content
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            # Initialize an empty string to store text
            text = ''
            
            # Loop through each page in the PDF
            for page in pdf.pages:
                # Extract text from each page and add it to the text string
                text += page.extract_text() + '\n'
            
            result = f"Url: {url}. Text: {text}"
            return result
    else:
        return "Failed to retrieve content."
    
def return_answer_to_expert(text: str) -> str:
    """When a satisfying answer is found, call this function to return your answer to the regulatory expert. """
    answer_found = True
    return text


def result_to_text(search: str) -> str:
    return f"Url: {search.url}. Title: {search.title}. Description: {search.description}"

def google_search(search_task: str) -> str:
    """Does a google search for the passed search term. Returns text, with the results divided by first, second etc. and containing the url, title and description. """
    results = list(search(search_task, num_results=5, advanced=True, safe=None))
    result_text = ""
    for i, result in enumerate(results, start=1):
        result_text += f'{str(i)}: {result_to_text(result)}. '

    g_searches.append(result_text)

    return result_text

In [127]:
google_search('What is the deadline to submit an article 46')

'1: Url: https://www.ema.europa.eu/en/human-regulatory-overview/post-authorisation/paediatric-medicines-post-authorisation/submitting-results-paediatric-studies. Title: Submitting results of paediatric studies. Description: 26 Feb 2024 — 1 . What is the “Article 46 paediatric study submission”? Rev. Oct 2023.. 2: Url: https://www.aifa.gov.it/sites/default/files/ped_qa_regulation_art45_46.pdf. Title: submission of paediatric studies according to articles 45 & 46. Description: The submission of all completed paediatric studies is regardless of their place of conduct of the trial. They should be submitted to each competent\xa0.... 3: Url: https://www.gov.uk/guidance/completed-paediatric-studies-submission-processing-and-assessment. Title: Completed Paediatric Studies - submission, processing .... Description: If the results of a paediatric study have been submitted to EMA or CMDh under Article 46 of Regulation (EC) 1901/2006 before 1 January 2021 or a decentralised\xa0.... 4: Url: https:/

## Set up the model

In this step you collate the functions into a "system" that is passed as `tools`, instantiate the model and start the chat session.

This block includes two options for interacting with the Gemini API. By toggling `use_sys_inst`, you can switch between using Gemini 1.5 Pro with a system instruction (highest quality but free-tier quota may be insufficient for a long chat session) or Gemini 1.0 Pro (higher free quota but does not support system instructions).

A retriable `send_message` function is also defined to help with low-quota conversations.

In [141]:
ordering_system = [google_search, return_answer_to_expert, get_text_from_pdf_url]

# Toggle this to switch between Gemini 1.5 with a system instruction, or Gemini 1.0 Pro.
use_sys_inst = True

model_name = 'gemini-1.5-flash' if use_sys_inst else 'gemini-1.0-pro'

model = genai.GenerativeModel(
    model_name, tools=ordering_system, system_instruction=GPT_PROMPT)

convo = model.start_chat(enable_automatic_function_calling=True)

@retry.Retry(initial=30)
def send_message(message):
  return convo.send_message(message)

In [142]:
import os
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [143]:
from IPython.display import display, Markdown

print('Welcome to EMA GPT!\n\n')

while not answer_found:
  response = send_message(input('> '))
  display(Markdown(response.text))

Welcome to EMA GPT!




The deadline to submit information on completed paediatric studies for authorised medicines is **6 months after the study is completed**. 

This is according to Article 46 of Regulation (EC) 1901/2006 and the EMA's guidelines on submitting results of paediatric studies.

[https://www.ema.europa.eu/en/human-regulatory-overview/post-authorisation/paediatric-medicines-post-authorisation/submitting-results-paediatric-studies](https://www.ema.europa.eu/en/human-regulatory-overview/post-authorisation/paediatric-medicines-post-authorisation/submitting-results-paediatric-studies) 


ValueError: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.

Some things to try:
* Ask about the menu (e.g. "what coffee drinks are available?")
* Use terms that are not specified in the prompt (e.g. "a strong latte" or "an EB tea")
* Change your mind part way through ("uhh cancel the latte sorry")
* Go off-menu ("a babycino")

## See also

This sample app showed you how to integrate a traditional software system (the coffee ordering functions) and an AI agent powered by the Gemini API. This is a simple, practical way to use LLMs that allows for open-ended human language input and output that feels natural, but still keeps a human in the loop to ensure correct operation.

To learn more about how Barista Bot works, check out:

* The [Barista Bot](https://aistudio.google.com/app/prompts/barista-bot) prompt
* [System instructions](../quickstarts/System_instructions.ipynb)
* [Automatic function calling](../quickstarts/Function_calling.ipynb)
